In [1]:
import pandas as pd
import copy
from fuzzywuzzy import fuzz
from pandasql import sqldf
pysqldf = lambda q:sqldf(q,locals())
global un_df
from functools import partial

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, locals())

/anaconda/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Load the Data

In [2]:
df = pd.read_csv('test_network.csv',encoding = "ISO-8859-1",dtype={'IncidentNumber':object})

### Column Names

In [3]:
df.head(2)

,Unnamed: 0,IncidentNumber,FirstName,LastName,FullName,InvolvementType
0,0,201600008,Sakip,Mofakmad,Sakip Mofakmad,WITNESS/OTHER
1,1,201600011,Nibic,Jikelez,Nibic Jikelez,WITNESS/OTHER


In [5]:
groups = df.groupby(['FullName'])

In [6]:
un_df = pd.DataFrame(list(groups.groups.keys()), columns = ['FullName'])

In [7]:
q1 = 'SELECT DISTINCT * FROM un_df'
ndf1 = sqldf(q1, globals())

In [8]:
ndf1 = ndf1.sort_values(['FullName'], ascending = True)

In [9]:
def get_ratio(row, target_name):
    name = row['FullName']
    return fuzz.token_sort_ratio(name, target_name)

#un_df['score'] = un_df.apply(get_ratio, axis=1)

In [10]:
def delete_name (row, target, DOB, un_df, MergeName):
    if target == 'None':
        return 
    index = row.name
    inner_index = un_df.index.get_loc(index)
    
    if inner_index - 30 < 0:
        Begin = 0
    else:
        Begin = inner_index - 30
    
    if inner_index + 30 > len(un_df)-1:
        End = len(un_df)-1
    else: 
        End = inner_index + 30
    
    un_df= un_df.iloc[Begin:End+1,]
    un_df2 = un_df[un_df['DOB'] == DOB]
    if un_df2.empty == False:
        #score = un_df1.apply(lambda x: get_ratio(x, target), axis = 1)
        un_df2['score'] = un_df2.apply(lambda x: get_ratio(x, target), axis = 1)
        un_df3 = un_df2[un_df2['score'] >= 80]

        un_df3 = un_df3.sort_values('FullName', ascending = False)
        
    
        if un_df3.empty == False:
           # print(un_df3)
            new_name = un_df3.iloc[0,0]
            # Get the index for the merged names
            merged_names = un_df3.index.tolist()
            MergeName[merged_names] = new_name

            #print(un_df.iloc[merged_names,])
            un_df.loc[merged_names, 'CheckName'] = 'None'
        
    
   # un_df1.drop(merged_names)

### Choose the Matching Range

In [11]:
un_df1 = ndf1.tail(200)
FullNames = un_df1['FullName']
un_df1['OriginName'] = un_df1['FullName']
un_df1['CheckName'] = un_df1['FullName']
MergeName = FullNames

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
un_df1.apply(lambda x: delete_name(x, x['CheckName'], x['DOB'],un_df1,MergeName), axis = 1)

In [13]:
un_df1 = un_df1.sort_values(['FullName'])

### Displaying Merged Names

In [14]:
q = 'SELECT * FROM un_df1 WHERE FullName in (SELECT FullName FROM un_df1 GROUP BY FullName HAVING COUNT(*) > 1)'

ndf = sqldf(q, globals())

In [15]:
ndf

,FullName,OriginName,CheckName


### Match the name in the original File

In [16]:
mydict = dict(zip(ndf.OriginName, ndf.FullName))

In [17]:
df["FullName"].replace(mydict, inplace=True)

In [18]:
df[df.FullName == 'Zynekwa Shantasia Williams']

,Unnamed: 0,IncidentNumber,FirstName,LastName,FullName,InvolvementType
